# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_key

'voc-2015200036168865441126268e0ed5be47456.24871461'

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
# [TODO] Define a query and run
query = "Lastest announcements of NVIDIA"
result = client.search(query)

In [5]:
result

{'query': 'Lastest announcements of NVIDIA',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'http://nvidianews.nvidia.com/news/latest',
   'title': 'Latest News | NVIDIA Newsroom',
   'content': '### NVIDIA and Samsung Build AI Factory to Transform Global Intelligent Manufacturing NVIDIA today announced that it is working with SK Group to build an AI factory to advance semiconductor research, development and production, as well as cloud infrastructure to support digital twin and AI agent development. ### NVIDIA and US Technology Leaders Unveil AI Factory Design to Modernize Government and Secure the Nation NVIDIA today announced a collaboration with Palantir Technologies Inc. to build a first-of-its-kind integrated technology stack for operational AI — including analytics capabilities, reference workflows, automation features and customizable, ... ### NVIDIA Launches Open Models and Data to Accelerate AI Innovation Across Language, Biology and Roboti

## Define Web Search tool

In [6]:
# [TODO] Define the search tool
@tool
def web_search(query: str) -> Dict:
    return client.search(query)

In [7]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [8]:
# [TODO] Agent with no tools
simple_agent = Agent('no_web_search', 'You are an assitant that gets the latest news')

In [9]:
# [TODO] Agent with web search tool
web_agent = Agent('web_search', 'You are an assitant that gets the latest news', tools)

In [10]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [11]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: voc-2015**************************************1461. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(run1.get_final_state()["messages"][-1].content)

In [ ]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

In [ ]:
print(run2.get_final_state()["messages"][-1].content)

**Web Agent**

In [ ]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

In [ ]:
print(run1.get_final_state()["messages"][-1].content)

In [ ]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

In [ ]:
print(run2.get_final_state()["messages"][-1].content)

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow